In [ ]:
import notebook_import_hook
notebook_import_hook.attach()
from helper.helper import *
import xgboost as xgb

In [ ]:
def xgbf_rename(s):
    '''Make feature name `str` s usable for XGBoost'''
    return s.replace('[', '{|').replace(']', '|}').replace('<', '(|').replace('>', '|)')

In [ ]:
def xgb_eval(bst, evals, i):
    '''Params - bst: `xgb.Booster`
                evals: list-like as passed to XGBoost eval_set param
                i: iteration number'''
    n = len(evals)
    line = bst.eval_set(evals, i)
    import re
    m = re.match(r'\[\-?\d+\]' + r'\t(\w+)\-[^\:]+\:(\S+)' * n, line)
    assert m and len(m.groups()) == n * 2, (line, m)
    fields, values = m.groups()[::2], tuple(float(x) for x in m.groups()[1::2])
    return O(**{f: v for f, v in zip(fields, values)}, _orig=line)

In [ ]:
def make_dmat(X, T, W, s, include='tr cv ho'):
    include = include.split() if isinstance(include, str) else include
    dmat = O()
    for se in include:
        dmat[se] = xgb.DMatrix(X[s[se]], T[s[se]], weight=W[s[se]], feature_names=[xgbf_rename(c) for c in X.columns])
    return dmat

In [6]:
from common import *

class BoostTransformer():
    def __init__(bt, ref=dh):
        bt.ref = ref
        bt.o = O()
        def_W(bt.o, ref)
        def_Y(bt.o, ref, include='scl std')
    
    @property
    def id(bt):
        class IdentityTransform():
            to_T = lambda self, Y, WY: (Y, WY)
            to_Y = lambda self, T: T
        return IdentityTransform()
    
    #@property
    #def zm(bt):
    #    class ZmeanTransform():
    #        def __init__(self):
    #            self.Y_wmean = bt.o.Y_wmeana.makeil_like(bt.ref)      
    #        to_T = lambda self, Y, WY: (Y - self.Y_wmean, WY)
    #        to_Y = lambda self, T: T + self.Y_wmean
    #    return ZmeanTransform()
    
    @property
    def scl(bt):
        class ScaleTransform():
            def __init__(self):
                self.Y_wzvar = bt.o.Y_wzvar.replace(0, 1).makeil_like(bt.ref)      
            to_T = lambda self, Y, WY: (Y / np.sqrt(self.Y_wzvar), WY * self.Y_wzvar)
            to_Y = lambda self, T: T * np.sqrt(self.Y_wzvar)
        return ScaleTransform()
    
    @property
    def std(bt):
        class StdevTransform():
            def __init__(self): 
                self.Y_wvar = bt.o.Y_wvar.replace(0, 1).makeil_like(bt.ref)      
            to_T = lambda self, Y, WY: (Y / np.sqrt(self.Y_wvar), WY * self.Y_wvar)
            to_Y = lambda self, T: T * np.sqrt(self.Y_wvar)
        return StdevTransform()
    
    @staticmethod
    def boost(Y0):
        class BoostTransform():
            def __init__(self):
                self.Y0 = Y0
            to_T = lambda self, Y, WY: (Y - self.Y0, WY)
            to_Y = lambda self, T: T + self.Y0
        return BoostTransform()

In [ ]:
if __name__ == '__main__':
    Y, WY = dh.y, dh.w
    bt = BoostTransformer()
    T, WT = bt.scl.to_T(dh.y, dh.w)
    T, WT = Y, WY
    print(np.sqrt((T ** 2 * WT).sum() / WT.sum()))
    print((T ** 2 * WT).sum() / WT.sum())